# 爬取猫眼电影排名(正则表达式)
为简化问题，使用镜像网站进行爬取[猫眼电影镜像](https://ssr1.scrape.center/)

单个电影的源代码如下：编写正则表达式进行数据整理

# 使用bs库

In [78]:
import json
import requests
from requests.exceptions import RequestException
import re
import time
from bs4 import BeautifulSoup

def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def parse_one_page(html):
    soup= BeautifulSoup(html,'xml')
    rows=soup.select(".el-card__body > .el-row")
    for row in rows :
        yield {
            'index': int(row.select_one(".name").get("href").split('/')[-1]),
            'image': row.select_one("img").get("src"),
            'title': row.select_one(".name").text.strip(),
            'categories': [span.text for span in row.select(".categories span") ],
            'time': [info.text for info in row.select(".info span")][-1],
            'score': float(row.select_one(".score").text.strip())
        }

def write_to_file(content):
    with open('result.txt', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')

In [79]:
def main(page):
    url = f"https://ssr1.scrape.center/page/{str(page)}"
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

In [80]:
for i in range(1,11):
    main(i)
    time.sleep(1)


{'index': 1, 'image': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c', 'title': '霸王别姬 - Farewell My Concubine', 'categories': ['剧情', '爱情'], 'time': '1993-07-26 上映', 'score': 9.5}
{'index': 2, 'image': 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@464w_644h_1e_1c', 'title': '这个杀手不太冷 - Léon', 'categories': ['剧情', '动作', '犯罪'], 'time': '1994-09-14 上映', 'score': 9.5}
{'index': 3, 'image': 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@464w_644h_1e_1c', 'title': '肖申克的救赎 - The Shawshank Redemption', 'categories': ['剧情', '犯罪'], 'time': '1994-09-10 上映', 'score': 9.5}
{'index': 4, 'image': 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@464w_644h_1e_1c', 'title': '泰坦尼克号 - Titanic', 'categories': ['剧情', '爱情', '灾难'], 'time': '1998-04-03 上映', 'score': 9.5}
{'index': 5, 'image': 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@464w_644h_1e_1c', 'title': '罗马假

In [67]:
int(row.select_one(".name").get("href").split('/')[-1])

11

In [49]:
row.select_one("img").get("src")

'https://p1.meituan.net/movie/06ec3c1c647942b1e40bca84036014e9490863.jpg@464w_644h_1e_1c'

In [70]:
row.select_one(".name").text.strip()

'V字仇杀队 - V for Vendetta'

In [46]:
[span.text for span in row.select(".categories span") ]

['剧情', '动作', '科幻', '惊悚']

In [54]:
[info.text for info in row.select(".info span")]

['美国、英国、德国', ' / ', '132 分钟', '2005-12-11 上映']

In [59]:
float(row.select_one(".score").text.strip())

8.9